# Step 1: Load data into dataframe

In [1]:
# from simiclass.data import load_10x_dataset, load_GSE_dataset
# p2data = <>
p2df = '/data/jianhao/hepatocyte_update_dataset_101619/magic_cell_mat_w_label_pd'

p2assignment = '/data/jianhao/hepatocyte_update_dataset_101619/hepa_assignment_PA249'
### [TODO] label_to_int = {cell_states: integer}
# load_dataset(p2data, p2df, p2fc, p2assignment)
p2label_convert_dict = '/data/jianhao/hepatocyte_update_dataset_101619/label_to_integer_dict_pickle'

label_to_int = {'PN14': 0,
               'Adult': 1,
               'PHx24': 2,
               'PHx48': 3,
               'PHx96': 4}
import pickle
with open(p2label_convert_dict, 'wb') as f:
    pickle.dump(label_to_int, f)

# Step 2: Perform simicLASSO and save weights

In [2]:
# from simiclasso.clus_regression import simicLASSO_op

k_cluster = 5
similarity = True
p2tf = '/home/jianhao2/simicLASSO_git/data/diff_gene_list/TF_ids_human_protein_atlas_pickle'
p2saved_file = '/data/jianhao/200TF_1000Target_w_r2_stderror_NF_100_l1e-2_l2e-5'
num_target_genes = 1000
num_TFs = 200

# num_rep = [number of repeated experiments]
# max_rcd_iter = 500000 # by default
# simicLASSO_op(p2df, p2fc, p2assignment, k_cluster, 
#     similarity, p2tf, p2saved_file, num_target_genes, gene_list_type,
#     num_rep, max_rcd_iter)

# Step 3: Evaluate weights and derived GRNs

In [3]:
%matplotlib inline

import pickle
import matplotlib.pyplot as plt
import numpy as np
import ipdb
import copy
plt.rcParams['figure.dpi'] = 200

In [4]:
# p2saved_file = '/data/jianhao/hepatocyte_update_dataset_101619/magic_data_True_similarity_1000_target'
# p2saved_file = '/data/jianhao/updated_dict_w_std_and_r2_pickle'
with open(p2saved_file, 'rb') as f:
    results_all = pickle.load(f)
    
results_all.keys()

dict_keys(['weight_dic', 'adjusted_r_squared', 'standard_error', 'TF_ids', 'query_targets'])

In [5]:
weight_dic = results_all['weight_dic']
TF_ids = results_all['TF_ids']
target_ids = results_all['query_targets']
print('length of TF: ',len(TF_ids))

for label in weight_dic:
    print(weight_dic[label].shape)
    break

length of TF:  200
(201, 1000)


In [6]:
# print(weight_dic)

In [7]:
def invert_dict(in_dict):
    out_dict = {}
    for k in in_dict.keys():
        val = in_dict[k]
        out_dict[val] = k
    return out_dict

with open(p2label_convert_dict, 'rb') as f:
    label2int_dict = pickle.load(f)

int2label_dict = invert_dict(label2int_dict)

# AUCell

In [8]:
import pandas as pd

In [9]:
original_df = pd.read_pickle(p2df)
original_df.columns = map(str.upper, original_df.columns)
original_df

,RP1,SOX17,MRPL15,LYPLA1,GM37988,TCEA1,ATP6V1H,RB1CC1,4732440D04RIK,PCMTD1,...,GM9465,GM31508,AC154311.1,FBXO40,AC168090.1,WFIKKN1,CRISP3,PELI3,BATF2,LABEL
PN14_AAACCCAAGAAATGGG,0.004854,0.000539,0.591115,0.366935,0.000315,0.239336,0.120199,0.113393,0.002072,0.323168,...,0.000111,-0.000066,0.000232,-1.373859e-05,0.0,0.0,0.0,0.0,0.000152,PN14
PN14_AAACCCAAGCTGAGTG,0.004455,0.001884,0.358050,0.271369,0.000951,0.142879,0.088292,0.108493,0.003443,0.247182,...,0.000028,0.000041,-0.000073,9.781208e-07,0.0,0.0,0.0,0.0,-0.000062,PN14
PN14_AAACCCACAACGTTAC,0.004492,0.001639,0.388067,0.285322,0.000816,0.155048,0.092182,0.108365,0.003189,0.256612,...,0.000021,0.000021,-0.000035,-2.123797e-06,0.0,0.0,0.0,0.0,-0.000038,PN14
PN14_AAACCCAGTGCAATAA,0.004453,0.001879,0.358323,0.271515,0.000947,0.143037,0.088365,0.108483,0.003443,0.247273,...,0.000027,0.000041,-0.000072,9.464348e-07,0.0,0.0,0.0,0.0,-0.000062,PN14
PN14_AAACCCAGTGCTCCGA,0.004463,0.001848,0.362035,0.273399,0.000929,0.144476,0.088859,0.108497,0.003409,0.248508,...,0.000026,0.000038,-0.000068,4.196861e-07,0.0,0.0,0.0,0.0,-0.000059,PN14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PHx96_TTTGGTTGTACCCGAC,-0.000623,0.000605,1.137205,0.697996,-0.000048,0.472497,0.185700,0.190232,-0.000664,0.583412,...,0.001488,0.000498,0.002821,1.312452e-03,0.0,0.0,0.0,0.0,0.001682,PHx96
PHx96_TTTGGTTTCCAGTGTA,0.000141,-0.000569,0.940362,0.768103,-0.000276,0.415704,0.176906,0.225551,0.003126,0.655176,...,0.000501,0.000726,0.002950,1.810040e-03,0.0,0.0,0.0,0.0,0.001591,PHx96
PHx96_TTTGGTTTCCATGATG,-0.000056,-0.000831,0.883037,0.738153,-0.000483,0.388767,0.159912,0.214883,0.002815,0.619076,...,0.000426,0.000728,0.002927,1.830409e-03,0.0,0.0,0.0,0.0,0.001548,PHx96
PHx96_TTTGTTGCAAGCTACT,0.003265,0.002349,0.657112,1.464797,-0.000965,0.585532,0.355755,0.569388,0.017105,1.287136,...,-0.000112,0.000185,0.001373,2.877550e-03,0.0,0.0,0.0,0.0,0.000824,PHx96


In [10]:
# step 1: normalized weight matrix by target expression.
print(target_ids[:3])
print(weight_dic[0].shape)

target_df = original_df[target_ids]
target_norms = np.linalg.norm(target_df, axis = 0)

['FDFT1', 'SS18L2', 'ERRFI1']
(201, 1000)


In [11]:
normalized_weights = {}
for label in weight_dic:
    normalized_mat = weight_dic[label] / target_norms
    normalized_weights[label] = normalized_mat[:len(TF_ids), :]
print(normalized_weights[0].shape)

(200, 1000)


In [12]:
# verified target expression normalization.
# a = np.array([[4, 2, 3], [8, 4, 6], [2, 1, 1.5]])
# b = [4, 2, 3]
# a / b

### array([[1. , 1. , 1. ],
###        [2. , 2. , 2. ],
###        [0.5, 0.5, 0.5]])

In [21]:
def cal_AUC(row_series_in, weight_series_in):
    df_new = pd.concat([row_series_in, row_series_in], axis = 1).T
    df_new = df_new.reset_index(drop = True)
    df_new.loc[1] = weight_series_in
    df_new.fillna(0)
#     ipdb.set_trace()
    df_sorted_by_row = df_new.sort_values(axis = 1, by = 0, ascending = False)
    len_of_genes = len(weight_series_in)
    running_sum = [np.sum(df_sorted_by_row.loc[1].values[:x+1]) for x in range(len_of_genes)]
    AUC_score = np.sum(running_sum) / (np.sum(weight_series_in) * len_of_genes)
    return AUC_score
    
    
    
def get_AUCell_mat(original_df, weight_dict, TF_ids, target_ids, threshold = None):
    '''
    weight_mat: # num_TFs * num_Targets.
    target_ids: all targets, thresholding should only change the value to 0, not removing.
    '''
    df_in = original_df.reset_index(drop = True)
    original_index = original_df.index
    AUC_dict = {}
    for label in weight_dict:
        weight_mat = np.abs(weight_dict[label])
        AUC_mat = np.zeros([df_in.shape[0], len(TF_ids)])
        for row in df_in.iterrows():
            tmp_AUC_row = np.zeros(len(TF_ids))
            row_idx = row[0]
            row_series_in = row[1][:len(TF_ids)]
            for tf_idx in range(len(TF_ids)):
                weight_series_in = pd.Series(data = weight_mat[tf_idx, :], index = target_ids)
                AUC_score = cal_AUC(row_series_in, weight_series_in)
                tmp_AUC_row[tf_idx] = AUC_score
            AUC_mat[row_idx, :] = tmp_AUC_row
            
        AUC_dict[label] = pd.DataFrame(data=AUC_mat, columns = TF_ids, index=original_index)
    
    return AUC_dict

In [ ]:
AUC_dict = get_AUCell_mat(original_df, normalized_weights, TF_ids, target_ids)

In [ ]:
AUC_dict

------
# Debugging

In [78]:
df_tmp = pd.DataFrame(data = np.random.rand(3, 4), columns = ['a', 'b', 'c', 'd'], index = ['x', 'y', 'z'])

df_test = df_tmp.reset_index(drop = True)
print(df_tmp)
print(df_test)
for row in df_test.iterrows():
    print(type(row[0]))
    print(row[1])

df_test.loc[1] = 0
test_seris = pd.Series([1, 2], index = ['a', 'c'])
df_test.loc[2] = test_seris
df_test.fillna(0)

df_test.sort_values(axis = 1, by = 0, ascending = False)

          a         b         c         d
x  0.933978  0.598364  0.597080  0.480695
y  0.400381  0.830637  0.868357  0.189374
z  0.323163  0.833104  0.046436  0.174509
          a         b         c         d
0  0.933978  0.598364  0.597080  0.480695
1  0.400381  0.830637  0.868357  0.189374
2  0.323163  0.833104  0.046436  0.174509
<class 'int'>
a    0.933978
b    0.598364
c    0.597080
d    0.480695
Name: 0, dtype: float64
<class 'int'>
a    0.400381
b    0.830637
c    0.868357
d    0.189374
Name: 1, dtype: float64
<class 'int'>
a    0.323163
b    0.833104
c    0.046436
d    0.174509
Name: 2, dtype: float64


,a,b,c,d
0,0.933978,0.598364,0.59708,0.480695
1,0.000000,0.000000,0.00000,0.000000
2,1.000000,NaN,2.00000,NaN


In [92]:
np.zeros(3)
df_test.loc[1] = [4, 0, 0, 0]

In [96]:
a = [1, 0, 0, 0]
np.sum([np.sum(df_test.fillna(0).loc[1].values[:x+1]) for x in range(4)]) / np.sum(df_test.fillna(0).loc[1]) / 4

1.0

In [94]:
df_test.fillna(0)

,a,b,c,d
0,0.933978,0.598364,0.59708,0.480695
1,4.000000,0.000000,0.00000,0.000000
2,1.000000,0.000000,2.00000,0.000000


In [ ]:
# test if notebook keep running after closing window, detach from tmux, and close terminal.
import time

def stopwatch(seconds):
    start = time.time()
    # time.time() returns the number of seconds since the unix epoch.
    # To find the time since the start of the function, we get the start
    # value, then subtract the start from all following values.
    time.clock()    
    # When you first call time.clock(), it just starts measuring
    # process time. There is no point assigning it to a variable, or
    # subtracting the first value of time.clock() from anything.
    # Read the documentation for more details.
    elapsed = 0
    while elapsed < seconds:
        elapsed = time.time() - start
        print("loop cycle time: %f, seconds count: %02d" % (time.clock() , elapsed)) 
        time.sleep(1)  
        # You were sleeping in your original code, so I've stuck this in here...
        # You'll notice that the process time is almost nothing.
        # This is because we are spending most of the time sleeping,
        # which doesn't count as process time.
        # For funsies, try removing "time.sleep()", and see what happens.
        # It should still run for the correct number of seconds,
        # but it will run a lot more times, and the process time will
        # ultimately be a lot more. 
        # On my machine, it ran the loop 2605326 times in 20 seconds.
        # Obviously it won't run it more than 20 times if you use time.sleep(1)

stopwatch(300)

loop cycle time: 0.640000, seconds count: 00
loop cycle time: 0.660000, seconds count: 01
loop cycle time: 0.660000, seconds count: 02
loop cycle time: 0.660000, seconds count: 03
loop cycle time: 0.660000, seconds count: 04
loop cycle time: 0.660000, seconds count: 05


### Conclusion: output from code will not be updated and stored.

# But the code will continue running and saved the variable.
-----

In [ ]:
from time import sleep
file = open("jupyter_tab_close_test", "w")
file.close()
i = 0
while i < 300:
    print("{}\n".format(i))
    i +=1
    with open("jupyter_tab_close_test", "a") as f:
        f.write("{}\n".format(i))
    sleep(1)

0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48



In [5]:
print(i)

300
